# 1. Imports & Setup

In [ ]:
import os, sys, json
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Optional: seaborn cho plot tổng quan (có thể bỏ)
import seaborn as sns

# Make project root current working dir
project_root = Path.cwd().parent
os.chdir(project_root)

print("Libraries imported successfully")
print(f"Project root: {project_root}")

In [ ]:
from src.config import (
    DATA_DIR, RESULTS_DIR, VISUALIZATIONS_DIR,
    MIN_INVENTORY_DAYS, MAX_INVENTORY_DAYS,
    GA_POPULATION_SIZE, GA_GENERATIONS,
    GA_CROSSOVER_PROB, GA_MUTATION_PROB,
    REQUIRED_DATA_FILES
)

from src.engine.analyzer import InventoryAnalyzer
from src.engine.rule_based import RuleBasedOptimizer
from src.engine.genetic_algorithm import GeneticAlgorithmOptimizer
from src.engine.results_manager import ResultsManager
from src.engine.prophet_forecaster import ProphetForecaster

print("Project components imported successfully")
print(f"Configuration loaded:")
print(f"   - Min inventory days: {MIN_INVENTORY_DAYS}")
print(f"   - Max inventory days: {MAX_INVENTORY_DAYS}")

# 2. Load and Check data

In [ ]:
data_dir = Path(project_root) / DATA_DIV if (DATA_DIV:=DATA_DIR) else Path(project_root) / DATA_DIR
results_dir = Path(project_root) / RESULTS_DIR

# Check required files
missing_files = [f for f in REQUIRED_DATA_FILES if not (data_dir / f).exists()]
if missing_files:
    print(f"Missing files: {missing_files}")
    print("Please run data generation first: python src/main.py --generate-data")
else:
    print("All required data files found")

print(f"\nData directory: {data_dir}")
for file in REQUIRED_DATA_FILES:
    p = data_dir / file
    if p.exists():
        print(f"   {file}: {p.stat().st_size/1024:.1f} KB")

print("\nLoading datasets...")
sales_df = pd.read_csv(data_dir / 'sales_data.csv')
sales_df['date'] = pd.to_datetime(sales_df['date'])
inventory_df = pd.read_csv(data_dir / 'inventory_data.csv')
inventory_df['store_id'] = inventory_df['store_id'].astype(int)
stores_df = pd.read_csv(data_dir / 'stores.csv')
products_df = pd.read_csv(data_dir / 'products.csv')
distance_matrix = pd.read_csv(data_dir / 'distance_matrix.csv', index_col=0)
transport_cost_matrix = pd.read_csv(data_dir / 'transport_cost_matrix.csv', index_col=0)
distance_matrix.index = distance_matrix.index.astype(int); distance_matrix.columns = distance_matrix.columns.astype(int)
transport_cost_matrix.index = transport_cost_matrix.index.astype(int); transport_cost_matrix.columns = transport_cost_matrix.columns.astype(int)

print(f"Sales Data: {len(sales_df):,} records")
print(f"Inventory Data: {len(inventory_df):,} records")
print(f"Stores Data: {len(stores_df):,} stores")
print(f"Products Data: {len(products_df):,} products")
print(f"Distance Matrix: {distance_matrix.shape[0]}x{distance_matrix.shape[1]}")
print(f"Transport Cost Matrix: {transport_cost_matrix.shape[0]}x{transport_cost_matrix.shape[1]}")
print(f"Sales date range: {sales_df['date'].min()} to {sales_df['date'].max()}")
print(f"Sales dtypes: date={sales_df['date'].dtype}, quantity={sales_df['quantity'].dtype}")

# Sanity schema for Prophet & inventory
required_sales_cols = {"date","store_id","product_id","quantity"}
missing_sales_cols = required_sales_cols - set(sales_df.columns)
if missing_sales_cols:
    raise ValueError(f"sales_df thiếu cột: {missing_sales_cols} (cần {sorted(list(required_sales_cols))})")

sales_df["store_id"]   = sales_df["store_id"].astype(int)
sales_df["product_id"] = sales_df["product_id"].astype(int)
sales_df["quantity"]   = pd.to_numeric(sales_df["quantity"], errors="coerce").fillna(0).astype(float)

possible_inv_cols = ["quantity","current_stock","stock_quantity","inventory_level"]
inventory_qty_col = next((c for c in possible_inv_cols if c in inventory_df.columns), None)
if not inventory_qty_col:
    raise ValueError(f"inventory_df thiếu cột tồn kho. Cần một trong {possible_inv_cols}")
inventory_df["product_id"] = inventory_df["product_id"].astype(int)
inventory_df[inventory_qty_col] = pd.to_numeric(inventory_df[inventory_qty_col], errors="coerce").fillna(0).astype(float)

# (Optional) quick peek
print("\nSample Data Overview:")
try:
    display(sales_df.head())
    display(inventory_df.head())
    display(stores_df.head())
    display(products_df.head())
except Exception:
    print(sales_df.head().to_string(index=False))
    print(inventory_df.head().to_string(index=False))


# 3. Quick EDA

In [ ]:
try:
    fig, axes = plt.subplots(2, 2, figsize=(14, 9))
    axes[0,0].hist(sales_df['quantity'], bins=30, alpha=0.7)
    axes[0,0].set_title('Sales Quantity Distribution')
    inv_col = inventory_qty_col
    axes[0,1].hist(inventory_df[inv_col], bins=30, alpha=0.7)
    axes[0,1].set_title(f'Inventory {inv_col} Distribution')
    if 'store_id' in sales_df.columns:
        sbs = sales_df.groupby('store_id')['quantity'].sum().sort_values(ascending=False).head(10)
        axes[1,0].bar(range(len(sbs)), sbs.values)
        axes[1,0].set_title('Top 10 Stores by Sales')
    ibs = inventory_df.groupby('store_id')[inv_col].sum().sort_values(ascending=False).head(10)
    axes[1,1].bar(range(len(ibs)), ibs.values)
    axes[1,1].set_title('Top 10 Stores by Inventory')
    plt.tight_layout(); plt.show()
except Exception as e:
    print(f"(Skipping EDA plots) Reason: {e}")

# 4. Analyzer

In [ ]:
print("\nInitializing Inventory Analyzer...")
analyzer = InventoryAnalyzer(
    sales_df=sales_df,
    inventory_df=inventory_df,
    stores=stores_df,
    products=products_df
)

print("Analyzing sales data...")
analysis_df = analyzer.analyze_sales_data()
try:
    display(analysis_df.head())
except:
    print(analysis_df.head().to_string(index=False))

print("Identifying inventory imbalances (legacy analyzer)...")
excess_df_legacy, needed_df_legacy = analyzer.identify_inventory_imbalances()
print(f"Legacy: excess rows={len(excess_df_legacy)}, needed rows={len(needed_df_legacy)}")


# 5. Prophet Forecasting → demand gaps (Prophet)

In [ ]:
print("\nForecasting demand with Prophet...")
forecaster = ProphetForecaster(
    weekly_seasonality=True,
    yearly_seasonality=True,
    daily_seasonality=False,
    holidays_df=None,                 # có thể nạp vn_holidays.csv với cột ['ds','holiday']
    seasonality_mode="multiplicative",
    interval_width=0.8
)

HORIZON_DAYS = 14
forecast_df = forecaster.fit_predict(
    sales_df=sales_df[["date","store_id","product_id","quantity"]],
    horizon_days=HORIZON_DAYS,
    regressors_df=None,               # nếu có regressors (promotion/price/discount...), nạp df vào đây
    freq="D",
    min_points_per_series=10
)

print(f"Forecast rows: {len(forecast_df):,}")
try:
    display(forecast_df.head())
except:
    print(forecast_df.head().to_string(index=False))

print("\nAggregating forecast to horizon demand...")
horizon_demand = (
    forecast_df.groupby(["store_id","product_id"], as_index=False)["yhat"]
               .sum()
               .rename(columns={"yhat":"forecast_units_horizon"})
)

stock_now = (
    inventory_df.groupby(["store_id","product_id"], as_index=False)[inventory_qty_col]
                .sum()
                .rename(columns={inventory_qty_col: "current_stock"})
)

balance = horizon_demand.merge(stock_now, on=["store_id","product_id"], how="left").fillna({"current_stock": 0})
balance["needed_units"] = (balance["forecast_units_horizon"] - balance["current_stock"]).clip(lower=0).round().astype(int)
balance["excess_units"] = (balance["current_stock"] - balance["forecast_units_horizon"]).clip(lower=0).round().astype(int)

needed_df_prophet = balance.loc[balance["needed_units"] > 0, ["store_id","product_id","needed_units"]].copy()
excess_df_prophet  = balance.loc[balance["excess_units"] > 0, ["store_id","product_id","excess_units"]].copy()

print(f"Prophet gaps: needed={len(needed_df_prophet)} rows (sum={int(needed_df_prophet['needed_units'].sum()):,}), "
      f"excess={len(excess_df_prophet)} rows (sum={int(excess_df_prophet['excess_units'].sum()):,})")


# 6. Transport cost matrix deep check (fix nan_cost_count)

In [ ]:
print("\nTRANSPORT COST MATRIX DEEP ANALYSIS")
print("=" * 50)
print(f"Matrix dimensions: {transport_cost_matrix.shape}")
print(f"\nMatrix sample (top-left 5x5):")
try:
    display(transport_cost_matrix.iloc[:5, :5])
except:
    print(transport_cost_matrix.iloc[:5, :5].to_string())

# Fix undefined var:
nan_cost_count = int(transport_cost_matrix.isna().sum().sum())
print(f"\nTotal NaN cost entries: {nan_cost_count}")
if nan_cost_count > 0:
    nan_mask = transport_cost_matrix.isna()
    total_diag = len(transport_cost_matrix.index)
    diagonal_nans = sum(nan_mask.loc[i, i] for i in transport_cost_matrix.index if i in transport_cost_matrix.columns)
    print(f"   Diagonal NaN entries: {diagonal_nans} / {total_diag}")
    print(f"   All NaN on diagonal? {'YES' if diagonal_nans == nan_cost_count else 'NO'}")

valid_vals = transport_cost_matrix.values.flatten()
valid_vals = valid_vals[~pd.isna(valid_vals)]
if valid_vals.size:
    print(f"\nValid Cost Stats: n={valid_vals.size}, min={valid_vals.min():,.0f}, "
          f"median={np.median(valid_vals):,.0f}, mean={valid_vals.mean():,.0f}, max={valid_vals.max():,.0f}")
print("=" * 50)


# 7. Init Optimizers

In [ ]:
print("\nInitializing Optimizers...")
rule_optimizer = RuleBasedOptimizer(
    distance_matrix=distance_matrix,
    transport_cost_matrix=transport_cost_matrix
)
ga_optimizer = GeneticAlgorithmOptimizer(
    distance_matrix=distance_matrix,
    transport_cost_matrix=transport_cost_matrix
)
print("Optimizers ready.")

# 8. Select Demand Gaps Source (Prophet vs Legacy)

In [ ]:
USE_PROPHET_DEMAND = True  # có thể tắt để so sánh với logic cũ

use_prophet = (
    USE_PROPHET_DEMAND
    and len(needed_df_prophet) > 0
    and len(excess_df_prophet) > 0
)
if use_prophet:
    print("Using Prophet-based needed/excess for optimization.")
    needed_src = needed_df_prophet.copy()
    excess_src = excess_df_prophet.copy()
else:
    print("Fallback to legacy analyzer-based needed/excess.")
    needed_src = needed_df_legacy.copy()
    excess_src = excess_df_legacy.copy()

for df_, cols in [(needed_src, ['store_id','product_id']),
                  (excess_src, ['store_id','product_id'])]:
    for c in cols:
        if c in df_.columns:
            df_[c] = df_[c].astype(int)

# Quick key check against cost matrix
missing_from_idx = ~excess_src['store_id'].isin(transport_cost_matrix.index)
missing_to_col   = ~needed_src['store_id'].isin(transport_cost_matrix.columns)
if missing_from_idx.any() or missing_to_col.any():
    print("⚠️ Some store_id not present in transport_cost_matrix index/columns.")
    print("   Missing FROM index:", sorted(excess_src.loc[missing_from_idx,'store_id'].unique().tolist()))
    print("   Missing TO   cols :", sorted(needed_src.loc[missing_to_col,'store_id'].unique().tolist()))

# 9. Run Optimizations (Rule-Based + GA)

In [ ]:
print("\nRunning Rule-Based Optimization (selected demand)...")
start_time = datetime.now()
rule_transfer_plan = rule_optimizer.optimize(excess_src, needed_src)
rule_duration = (datetime.now() - start_time).total_seconds()

print("Running Genetic Algorithm Optimization (selected demand)...")
start_time = datetime.now()
ga_pl = ga_optimizer.optimize(
    excess_inventory=excess_src,
    needed_inventory=needed_src,
    population_size=GA_POPULATION_SIZE,
    num_generations=GA_GENERATIONS,
    crossover_prob=GA_CROSSOVER_PROB,
    mutation_prob=GA_MUTATION_PROB,
    tournament_size=3,
    verbose=True
)
ga_transfer_plan = ga_pl if ga_pl is not None else pd.DataFrame()
ga_duration = (datetime.now() - start_time).total_seconds()

# 10. Summaries & Diagnostics

In [ ]:
def summarize_plan(name, df):
    if df is None or len(df) == 0:
        print(f"{name}: No transfers generated.")
        return None
    total_transfers = len(df)
    total_units = int(df['units'].sum())
    total_cost  = float(df['transport_cost'].sum())
    avg_cost    = (total_cost / total_units) if total_units > 0 else 0.0
    print(f"{name}: transfers={total_transfers}, units={total_units:,}, "
          f"cost={total_cost:,.0f} VND, avg/unit={avg_cost:,.0f} VND")
    return dict(name=name, transfers=total_transfers, units=total_units, cost=total_cost, avg=avg_cost)

print("\n=== Results Summary ===")
rbm = summarize_plan("Rule-Based", rule_transfer_plan)
gam = summarize_plan("Genetic Algorithm", ga_transfer_plan)
print(f"Durations: Rule-Based={rule_duration:.2f}s, GA={ga_duration:.2f}s")

for nm, plan in [("Rule-Based", rule_transfer_plan), ("Genetic", ga_transfer_plan)]:
    if plan is not None and len(plan):
        zeros = int((plan['transport_cost'] == 0).sum())
        if zeros:
            print(f"⚠️ {nm}: {zeros} rows have transport_cost=0 → check cost matrix & store_id mapping.")

# Pick best by minimal total cost
best_name, best_plan = None, None
best_cost = float('inf')
for nm, pl in [("Rule-Based", rule_transfer_plan), ("Genetic Algorithm", ga_transfer_plan)]:
    if pl is not None and len(pl):
        c = pl['transport_cost'].sum()
        if c < best_cost:
            best_cost, best_name, best_plan = c, nm, pl

if best_plan is not None:
    print(f"\nBest: {best_name} | cost={best_cost:,.0f} VND | transfers={len(best_plan)} | units={int(best_plan['units'].sum()):,}")
    try:
        display(best_plan.head(10))
    except:
        print(best_plan.head(10).to_string(index=False))
else:
    print("\nNo valid plan produced. Check inputs & cost matrix.")


# 11. Impact Analysis (using analyzer)

In [ ]:
if best_plan is not None and len(best_plan):
    print(f"\nAnalyzing Impact of {best_name} plan...")
    post_transfer_impact, post_analysis = analyzer.evaluate_plan_impact(best_plan)
    try:
        display(post_transfer_impact)
    except:
        print(post_transfer_impact.to_string(index=False))

# 12. Final Summary & Save

In [ ]:
print("\nFINAL OPTIMIZATION SUMMARY")
print("=" * 50)
print(f"   • Total stores: {stores_df['store_id'].nunique()}")
print(f"   • Total products: {products_df['product_id'].nunique()}")
print(f"   • Prophet Needed: {len(needed_df_prophet)} ({int(needed_df_prophet['needed_units'].sum()):,})")
print(f"   • Prophet Excess : {len(excess_df_prophet)} ({int(excess_df_prophet['excess_units'].sum()):,})")
if rbm:
    print(f"   • Rule-Based: cost={rbm['cost']:,.0f}, units={rbm['units']:,.0f}, transfers={rbm['transfers']}")
if gam:
    print(f"   • GA        : cost={gam['cost']:,.0f}, units={gam['units']:,.0f}, transfers={gam['transfers']}")
if best_plan is not None:
    print(f"   • Recommended: {best_name} (cost={best_cost:,.0f})")

# Save results
save_results = True
if save_results and best_plan is not None:
    results_dir.mkdir(exist_ok=True)
    best_plan_path = results_dir / 'notebook_best_transfer_plan.csv'
    best_plan.to_csv(best_plan_path, index=False)
    print(f"\nSaved best plan → {best_plan_path}")

    # Comparison table if both exist
    comp = []
    if rbm: comp.append({"Algorithm":"Rule-Based", **{k:v for k,v in rbm.items() if k!='name'}})
    if gam: comp.append({"Algorithm":"Genetic Algorithm", **{k:v for k,v in gam.items() if k!='name'}})
    if comp:
        comp_df = pd.DataFrame(comp)
        comp_path = results_dir / 'notebook_algorithm_comparison.csv'
        comp_df.to_csv(comp_path, index=False)
        print(f"Saved comparison → {comp_path}")

    summary = {
        'timestamp': datetime.now().isoformat(),
        'best_algorithm': best_name,
        'total_cost': float(best_cost),
        'total_transfers': int(len(best_plan)),
        'total_units': int(best_plan['units'].sum()),
        'prophet_excess_rows': int(len(excess_df_prophet)),
        'prophet_needed_rows': int(len(needed_df_prophet))
    }
    summary_path = results_dir / 'notebook_analysis_summary.json'
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)
    print(f"Saved summary → {summary_path}")

print("\nAnalysis Complete!")